In [6]:
# PART : Data Preparation

import os
import re
import pandas as pd
import numpy as np


SENTIMENT_PATH = "fear_greed_index.csv"
TRADER_PATH    = "historical_data.csv"
os.makedirs("outputs", exist_ok=True)

# ------- Load files -------
if not os.path.exists(SENTIMENT_PATH):
    raise FileNotFoundError(f"Sentiment file not found at: {SENTIMENT_PATH}")
if not os.path.exists(TRADER_PATH):
    raise FileNotFoundError(f"Trades file not found at: {TRADER_PATH}")

sent   = pd.read_csv(SENTIMENT_PATH)
trades = pd.read_csv(TRADER_PATH, low_memory=False)

print("Loaded: sentiment rows =", len(sent), ", trades rows =", len(trades))

# ------- Document shapes / missing / duplicates -------
def doc_df(df, name, n_show=3):
    print(f"\n-- {name} --")
    print("shape:", df.shape)
    print("columns:", df.columns.tolist())
    print("missing (top 10):")
    print(df.isnull().sum().sort_values(ascending=False).head(10).to_string())
    print("duplicate rows:", df.duplicated().sum())
    print(df.head(n_show).to_string())   # FIX: replaced display() with print()

doc_df(sent,   "Sentiment (raw)")
doc_df(trades, "Trades (raw)")

# ------- Normalize column names -------
sent.columns   = sent.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)
trades.columns = (trades.columns
                  .str.strip()
                  .str.replace('\xa0', ' ', regex=False)
                  .str.strip()
                  .str.replace(r'\s+', '_', regex=True)
                  .str.lower())

print("\nNormalized sentiment cols:", list(sent.columns))
print("Normalized trades cols:",    list(trades.columns))

# ------- Detect sentiment columns -------
sent_date_col  = next((c for c in sent.columns if "date"  in c), None)
sent_label_col = next((c for c in sent.columns
                       if any(k in c for k in ["class","sent","classification"])), None)
if sent_date_col is None or sent_label_col is None:
    raise ValueError("Couldn't detect sentiment date/class columns. Found: " + ", ".join(sent.columns))

sent['date']      = pd.to_datetime(sent[sent_date_col], errors='coerce').dt.date
sent['sentiment'] = sent[sent_label_col].astype(str).str.strip().str.capitalize()
sent = sent[['date', 'sentiment']].dropna().drop_duplicates().reset_index(drop=True)
print("\nSentiment normalized: rows =", len(sent),
      "| date range:", sent['date'].min(), "->", sent['date'].max())

# ------- Detect trade columns -------
def pick(cols):
    for c in cols:
        if c in trades.columns:
            return c
    return None

acct_col        = pick(['account','acct','user','client'])
timestamp_col   = pick(['timestamp','timestamp_ist','timestamp_utc','time','ts','datetime'])
closed_pnl_col  = next((c for c in trades.columns if 'closed' in c and 'pnl' in c), None) \
                  or pick(['closed_pnl','realized_pnl','pnl','profit'])
size_usd_col    = pick(['size_usd','sizeusd','size_usd.'])
size_tokens_col = pick(['size_tokens','size_token'])
size_col_any    = size_usd_col or size_tokens_col or pick(['size','qty','quantity'])
side_col        = pick(['side','direction','trade_side'])
leverage_col    = pick(['leverage','lev','leverage_ratio','start_position','margin'])

print("\nDetected trade columns:")
for k, v in [("account", acct_col), ("timestamp", timestamp_col),
              ("closed_pnl", closed_pnl_col), ("size_usd", size_usd_col),
              ("size_tokens", size_tokens_col), ("any_size", size_col_any),
              ("side", side_col), ("leverage", leverage_col)]:
    print(f"  {k}: {v}")

required_missing = [r for r in [acct_col, timestamp_col, closed_pnl_col] if r is None]
if required_missing:
    raise ValueError(f"Missing required columns: {required_missing}. Available: {list(trades.columns)}")

# ------- Convert numeric fields safely -------
trades[closed_pnl_col] = pd.to_numeric(trades[closed_pnl_col], errors='coerce')
if size_col_any:
    trades[size_col_any] = pd.to_numeric(trades[size_col_any], errors='coerce')
if leverage_col and leverage_col in trades.columns:
    trades[leverage_col] = pd.to_numeric(trades[leverage_col], errors='coerce')

# ------- Robust timestamp parsing -------
def try_parse_epoch_unit(series):
    """Try units ns/us/ms/s; return (parsed_series, unit) or (None, None)."""
    snum = pd.to_numeric(series, errors='coerce').dropna()
    if len(snum) == 0:
        return None, None
    sample = snum.sample(min(len(snum), 200), random_state=1)
    best_unit, best_score = None, -1
    for u in ['ns', 'us', 'ms', 's']:
        try:
            # FIX: cast to plain int64 numpy array to avoid overflow with Int64 nullable
            dt_sample = pd.to_datetime(sample.values.astype('int64'), unit=u, errors='coerce')
            score = pd.Series(dt_sample).dt.year.between(2009, 2035).sum()
            if score > best_score:
                best_unit, best_score = u, score
        except Exception:
            continue
    if best_score <= 0:
        return None, None
    try:
        parsed = pd.to_datetime(
            pd.to_numeric(series, errors='coerce').values.astype('float64').astype('int64'),
            unit=best_unit, errors='coerce'
        )
        return pd.Series(parsed, index=series.index), best_unit
    except Exception:
        return None, None

def clean_timestamp_strings(s):
    s = s.astype(str).str.strip()
    s = s.str.replace(r'\s+[A-Z]{2,4}$', '', regex=True)  # strip trailing TZ abbreviations
    s = s.str.replace(',', ' ', regex=False)
    return s

# Primary: numeric epoch detection
parsed_dt, unit_used = try_parse_epoch_unit(trades[timestamp_col])
if parsed_dt is None or parsed_dt.isna().all():
    cleaned   = clean_timestamp_strings(trades[timestamp_col])
    parsed_dt = pd.to_datetime(cleaned, errors='coerce', utc=False)
    method    = "string_clean_parse"
else:
    method = f"epoch_unit_{unit_used}"

# Fallback: try alternate timestamp columns
if parsed_dt.isna().sum() >= len(parsed_dt) * 0.99:
    for alt in ['timestamp_ist', 'timestamp_utc', 'time', 'datetime']:
        if alt in trades.columns and alt != timestamp_col:
            parsed_alt, alt_unit = try_parse_epoch_unit(trades[alt])
            if parsed_alt is not None and parsed_alt.notna().sum() > 0:
                parsed_dt, method = parsed_alt, f"alt_epoch_{alt}_{alt_unit}"
                break
            parsed_alt = pd.to_datetime(
                clean_timestamp_strings(trades[alt]), errors='coerce', utc=False)
            if parsed_alt.notna().sum() > 0:
                parsed_dt, method = parsed_alt, f"alt_string_{alt}"
                break

trades['datetime'] = parsed_dt
print(f"\nTimestamp parsing method: {method}")
print("Parsed non-null:", trades['datetime'].notna().sum(), "of", len(trades))
if trades['datetime'].notna().sum() > 0:
    print("Sample datetimes:", trades['datetime'].head(5).tolist())
    print("Range:", trades['datetime'].min(), "->", trades['datetime'].max())

# Fix: re-parse if >40% of dates are 1970 (epoch unit mismatch)
if trades['datetime'].notna().sum() > 0:
    n_1970   = (trades['datetime'].dt.year == 1970).sum()
    frac_1970 = n_1970 / max(1, trades['datetime'].notna().sum())
    if frac_1970 > 0.4:
        print(f"  [WARN] {n_1970} rows are year 1970 ({frac_1970:.1%}); re-parsing with best numeric unit...")
        snum = pd.to_numeric(trades[timestamp_col], errors='coerce').dropna()
        if len(snum) > 0:
            best_unit, best_valid = None, -1
            for u in ['ns', 'us', 'ms', 's']:
                try:
                    dt_try = pd.to_datetime(snum.values.astype('int64'), unit=u, errors='coerce')
                    valid  = pd.Series(dt_try).dt.year.between(2009, 2035).sum()
                    if valid > best_valid:
                        best_unit, best_valid = u, valid
                except Exception:
                    continue
            if best_unit:
                trades['datetime'] = pd.to_datetime(
                    pd.to_numeric(trades[timestamp_col], errors='coerce')
                      .values.astype('float64').astype('int64'),
                    unit=best_unit, errors='coerce'
                )
                print("  Reparsed unit:", best_unit, "| valid count:", best_valid)
                print("  New range:", trades['datetime'].min(), "->", trades['datetime'].max())

trades['date']  = trades['datetime'].dt.date
before_drop     = len(trades)
trades          = trades.dropna(subset=['date']).reset_index(drop=True)
print(f"Dropped {before_drop - len(trades)} rows (invalid datetime). Remaining: {len(trades)}")
print("Trades date range:", trades['date'].min(), "->", trades['date'].max())

# ------- Rename to canonical column names -------
rename_map = {}
if acct_col        and acct_col        != 'account':     rename_map[acct_col]        = 'account'
if closed_pnl_col  and closed_pnl_col  != 'closed_pnl':  rename_map[closed_pnl_col]  = 'closed_pnl'
if size_usd_col    and size_usd_col    != 'size_usd':    rename_map[size_usd_col]    = 'size_usd'
if size_tokens_col and size_tokens_col != 'size_tokens': rename_map[size_tokens_col] = 'size_tokens'
if size_col_any    and size_col_any not in ['size_usd','size_tokens','size']:
    rename_map[size_col_any] = 'size'
if side_col        and side_col        != 'side':        rename_map[side_col]        = 'side'
if leverage_col    and leverage_col in trades.columns and leverage_col != 'leverage':
    rename_map[leverage_col] = 'leverage'

trades.rename(columns=rename_map, inplace=True)
canonical = [c for c in ['account','closed_pnl','size_usd','size_tokens','size',
                          'datetime','date','side','leverage'] if c in trades.columns]
print("\nCanonical columns present:", canonical)

# ------- Indicators -------
trades['is_win'] = trades['closed_pnl'] > 0
if   'size_usd'    in trades.columns: trades['abs_size'] = trades['size_usd'].abs()
elif 'size_tokens' in trades.columns: trades['abs_size'] = trades['size_tokens'].abs()
elif 'size'        in trades.columns: trades['abs_size'] = trades['size'].abs()
else:                                  trades['abs_size'] = np.nan

# ------- Daily per-account aggregation -------
agg_dict = {
    'daily_pnl':       ('closed_pnl', 'sum'),
    'trades_count':    ('closed_pnl', 'count'),
    'win_count':       ('is_win',     'sum'),
    'avg_trade_size':  ('abs_size',   'mean'),
}
if 'leverage' in trades.columns:
    agg_dict['avg_leverage'] = ('leverage', 'mean')

daily_account = trades.groupby(['date', 'account']).agg(**agg_dict).reset_index()

# Long / short counts
if 'side' in trades.columns:
    def count_side(s, positives):
        return s.astype(str).str.lower().isin(positives).sum()

    longs  = (trades.groupby(['date','account'])['side']
              .apply(lambda s: count_side(s, {'buy','long'}))    # FIX: no include_groups needed
              .reset_index(name='long_count'))
    shorts = (trades.groupby(['date','account'])['side']
              .apply(lambda s: count_side(s, {'sell','short'}))
              .reset_index(name='short_count'))
    daily_account = daily_account.merge(longs,  on=['date','account'], how='left')
    daily_account = daily_account.merge(shorts, on=['date','account'], how='left')
else:
    daily_account['long_count']  = np.nan
    daily_account['short_count'] = np.nan

daily_account['win_rate']         = daily_account['win_count']  / daily_account['trades_count']
daily_account['long_short_ratio'] = daily_account['long_count'] / (daily_account['short_count'] + 1e-9)

# Intra-day drawdown proxy
def day_drawdown(sub):
    sub  = sub.sort_values('datetime')
    csum = sub['closed_pnl'].cumsum()
    return csum.min() if not csum.empty else np.nan

# FIX: use include_groups=False to suppress FutureWarning
dd = (trades.groupby(['date','account'])
      .apply(day_drawdown, include_groups=False)     # pandas ≥ 2.2 compat
      .reset_index(name='daily_min_cum_pnl'))
daily_account = daily_account.merge(dd, on=['date','account'], how='left')

# ------- Platform / day aggregation -------
platform_agg = {
    'total_pnl':      ('daily_pnl',        'sum'),
    'avg_win_rate':   ('win_rate',          'mean'),
    'avg_trade_size': ('avg_trade_size',    'mean'),
    'total_trades':   ('trades_count',      'sum'),
    'avg_long_short': ('long_short_ratio',  'mean'),
}
if 'avg_leverage' in daily_account.columns:
    platform_agg['avg_leverage'] = ('avg_leverage', 'mean')

daily = daily_account.groupby('date').agg(**platform_agg).reset_index()
daily = daily.merge(sent, on='date', how='left')

# ------- Print Part A Summary -------
print("\n--- Part A Summary ---")
print("Trades:          rows,cols:", trades.shape)
print("Daily_account:   rows,cols:", daily_account.shape)
print("Daily (platform):rows,cols:", daily.shape)

print("\nMissing values (trades) top 10:")
print(trades.isnull().sum().sort_values(ascending=False).head(10).to_string())

print("\nMissing values (daily_account) top 10:")
print(daily_account.isnull().sum().sort_values(ascending=False).head(10).to_string())

print("\nDuplicates: trades =",        trades.duplicated().sum())
print("Duplicates: daily_account =",  daily_account.duplicated().sum())

if 'leverage' in trades.columns:
    print("\nLeverage summary (trades):")
    print(trades['leverage'].describe().to_string())

acct_summary = (daily_account.groupby('account')
                .agg(total_pnl=('daily_pnl','sum'),
                     total_trades=('trades_count','sum'),
                     avg_win_rate=('win_rate','mean'))
                .reset_index())
print("\nTop 5 accounts by total_pnl:")
print(acct_summary.sort_values('total_pnl', ascending=False).head(5).to_string(index=False))

# ------- Save outputs -------
daily_account.to_csv("outputs/daily_account_metrics.csv", index=False)
daily.to_csv("outputs/daily_metrics.csv",         index=False)
trades.to_csv("outputs/trades_cleaned.csv",        index=False)

print("\nSaved to outputs/: daily_account_metrics.csv, daily_metrics.csv, trades_cleaned.csv")
print("\n✅ PART A COMPLETE — Share your Part B code when ready.")
# ============================================================
# END PART A (FIXED)
# ============================================================

Loaded: sentiment rows = 2644 , trades rows = 211224

-- Sentiment (raw) --
shape: (2644, 4)
columns: ['timestamp', 'value', 'classification', 'date']
missing (top 10):
timestamp         0
value             0
classification    0
date              0
duplicate rows: 0
    timestamp  value classification        date
0  1517463000     30           Fear  2018-02-01
1  1517549400     15   Extreme Fear  2018-02-02
2  1517635800     40           Fear  2018-02-03

-- Trades (raw) --
shape: (211224, 16)
columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
missing (top 10):
Account            0
Coin               0
Execution Price    0
Size Tokens        0
Size USD           0
Side               0
Timestamp IST      0
Start Position     0
Direction          0
Closed PnL         0
duplicate rows: 0
                                   